In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.

In [2]:
# xData => [공부 시간, 과외 시간]
temp = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
print(temp)
print(type(temp))

xData = np.array(temp)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
# yData => [합격 여부] => 실제값
temp = [0, 0, 0, 1, 1, 1, 1]
print(temp)
print(type(temp))

yData = np.array(temp)
print(yData)
# reshape() : 넘파이에서 데이터는 그대로 유지한 채 배열의 형태(차원)을 변경한다. 1행 7열 넘파이 배열 => 7행 1열 넘파이 배열
yData = np.array(temp).reshape(7, 1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해 대입되는 넘파이 배열의 차원을 지정해줘야 한다.
# [None, 2] => placeholder에 대입되는 넘파이 배열의 행의 개수는 몇개라도 상관이 없고 열의 개수는 무조건 2개이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

기울기(가중치) a와 y절편(바이어스) b를 임의로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2, 1]), dtype=tf.float32) # 난수를 2행 1열로 만든다.
b = tf.Variable(tf.random_uniform([1]), dtype=tf.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a: {}'.format(sess.run(a)))
print('b: {}'.format(sess.run(b)))
print('a1: {}, a2: {}, b: {}'.format(sess.run(a)[0], sess.run(a)[1], sess.run(b)))

a: [[0.23371518]
 [0.92461264]]
b: [0.346743]
a1: [0.23371518], a2: [0.92461264], b: [0.346743]


시그모이드 방정식, 오차 함수, 경사 하강

In [6]:
# matmul() : tensorflow에서 행렬의 곱셈을 계산하는 메소드
# sigmoid() : tensorflow에서 시그모이드 방정식을 계산하는 메소드
y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강

sigmoid() 메소드의 실행 결과(y)가 0.5 이상이면 1을 0.5 미만이면 0을 리턴한다.

In [7]:
sess = tf.Session()
# cast(캐스팅할 데이터, dtype=캐스팅될 데이터 타입) : tensorflow에서 형변환을 실행하는 메소드
# predicted = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
# print(sess.run(predicted)) # [1 2]
# cast() 메소드의 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 캐스팅 한다.
predicted = tf.cast(0.4 >= 0.5, dtype=tf.float32)
print(sess.run(predicted)) # 0.0
predicted = tf.cast(0.5 >= 0.5, dtype=tf.float32)
print(sess.run(predicted)) # 1.0

0.0
1.0


sigmoid() 메소드를 실행한 예상값을 계산한다.

In [8]:
predicted = tf.cast(y >= 0.5, dtype=tf.float32) # 예측값, sigmoid() 메소드를 실행한 결과(y)를 0 또는 1로 변환한다.
# 예측값과 실제값이 일치하는 정도(정확도)를 계산한다.
# equal() : tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False를 리턴하는 메소드
# equal() 메소드로 예측값(predicted)과 실제값(Y)이 같은가 비교한 후 True 또는 False 결과를 1 또는 0으로 캐스팅하고 시행된 전체
# 결과의 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

학습을 시킨다.

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(3001):
    # sess.run([기울기, y절편, 오차함수, 경사하강], feed_fict={X: 입력 데이터, Y: 입력 데이터에 따른 실제값})
    a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict={X: xData, Y: yData})
    if i % 300 == 0:
        # Epoch: 학습 횟수, a1: 공부 시간 기울기, a2: 과외 시간 기울기, b: y절편, loss: 오차함수
        print('Epoch: %4d, a1: %9.5f, a2: %9.5f, b: %9.5f, loss: %9.5f' % (i, a_[0], a_[1], b_, loss_))
        
# 테스트 데이터를 만든다.
# new_x = np.array([6, 5]) # 테스트 데이터가 1차원 넘파이 배열이기 때문에 행렬의 곱셈 연산이 실행되지 않는다.
# print(new_x) # [6 5] => 1차원
# reshape() 메소드를 이용해서 1행 2열인 2차원 넘파이 배열로 테스트 데이터를 만들어야 한다.
new_x = np.array([6, 5]).reshape(1, 2)
# print(new_x) # [[6 5]] => 2차원

# sigmoid() 메소드의 연산 결과에 테스트 데이터를 대입해서 연산한다.
result, new_y = sess.run([predicted, y], feed_dict={X: new_x})
print('공부 시간: %d, 과외 시간: %d' % (new_x[:, 0], new_x[:, 1]))
print('합격 여부: %d, 합격 확률: %f' % (result, new_y))
print('=' * 80)

for i in range(11):
    new_x = np.array([0, i]).reshape(1, 2)
    result, new_y = sess.run([predicted, y], feed_dict={X: new_x})
    print('공부 시간: %d, 과외 시간: %2d' % (new_x[:, 0], new_x[:, 1]), end=' ')
    print('합격 여부: %s, 합격 확률: %7.4f%%' % ('  합격' if result == 1 else '불합격', new_y * 100))

Epoch:    0, a1:   0.01480, a2:   0.28791, b:   0.40137, loss:   1.11868
Epoch:  300, a1:   0.78895, a2:  -0.50906, b:  -2.47336, loss:   0.26414
Epoch:  600, a1:   0.79802, a2:  -0.25166, b:  -3.93369, loss:   0.18960
Epoch:  900, a1:   0.71452, a2:   0.06558, b:  -4.99058, loss:   0.14851
Epoch: 1200, a1:   0.61558, a2:   0.36455, b:  -5.82740, loss:   0.12175
Epoch: 1500, a1:   0.52012, a2:   0.63227, b:  -6.52194, loss:   0.10292
Epoch: 1800, a1:   0.43322, a2:   0.86916, b:  -7.11617, loss:   0.08899
Epoch: 2100, a1:   0.35566, a2:   1.07881, b:  -7.63569, loss:   0.07831
Epoch: 2400, a1:   0.28683, a2:   1.26521, b:  -8.09733, loss:   0.06988
Epoch: 2700, a1:   0.22569, a2:   1.43198, b:  -8.51278, loss:   0.06306
Epoch: 3000, a1:   0.17119, a2:   1.58221, b:  -8.89052, loss:   0.05745
공부 시간: 6, 과외 시간: 5
합격 여부: 1, 합격 확률: 0.511920
공부 시간: 0, 과외 시간:  0 합격 여부: 불합격, 합격 확률:  0.0138%
공부 시간: 0, 과외 시간:  1 합격 여부: 불합격, 합격 확률:  0.0669%
공부 시간: 0, 과외 시간:  2 합격 여부: 불합격, 합격 확률:  0.3249%
공부 시간: 0